In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sktime.classification.kernel_based import ROCKETClassifier

In [28]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [29]:
train_df.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING


In [30]:
train_df.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515,17.413085
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122,8.975143
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414,8.000000
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181,19.000000
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659,26.000000
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000,30.000000


In [31]:
le = LabelEncoder()
le.fit(train_df['Activity'])
le.classes_

array(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [32]:
train_df['class'] = le.transform(train_df['Activity'])
train_df = train_df.drop(columns=['Activity', 'subject'])

test_df['class'] = le.transform(test_df['Activity'])
test_df = test_df.drop(columns=['Activity', 'subject'])

In [33]:
X_train, y_train = train_df.drop(columns='class'), train_df['class']
X_test, y_test = test_df.drop(columns='class'), test_df['class']

feature_cols = X_test.columns

In [34]:
help(ROCKETClassifier.fit)

Help on function fit in module sktime.classification.base:

fit(self, X, y)
    Fit time series classifier to training data.
    
    Parameters
    ----------
    X : 3D np.array, array-like or sparse matrix
            of shape = [n_instances,n_dimensions,series_length]
            or shape = [n_instances,series_length]
        or single-column pd.DataFrame with pd.Series entries
    y : array-like, shape =  [n_instances] - the class labels.
    
    Returns
    -------
    self :
        Reference to self.
    
    Notes
    -----
    Changes state by creating a fitted model that updates attributes
    ending in "_" and sets is_fitted flag to True.



In [35]:
X_train = np.expand_dims(X_train.to_numpy(), axis=1)
X_test = np.expand_dims(X_test.to_numpy(), axis=1)

In [36]:
X_train.shape, X_test.shape

((7352, 1, 561), (2947, 1, 561))

In [37]:
%%time
clf = ROCKETClassifier(num_kernels=50, n_jobs=10)
clf.fit(X_train, y_train)

CPU times: user 2.99 s, sys: 1.12 s, total: 4.1 s
Wall time: 461 ms


ROCKETClassifier(n_jobs=10, num_kernels=50, random_state=None)

In [38]:
%%time
y_pred = clf.predict(X_test) 
y_pred_prob = clf.predict_proba(X_test) 

CPU times: user 2.66 s, sys: 760 ms, total: 3.42 s
Wall time: 356 ms


In [39]:
accuracy_score(y_test, y_pred)

0.8568035290125552

In [40]:
y_test[y_test != y_pred]

31      1
33      1
34      1
35      1
38      1
       ..
2939    5
2943    5
2944    5
2945    5
2946    5
Name: class, Length: 422, dtype: int64

In [41]:
f1_score(y_test, y_pred, average='weighted')

0.8552106150416815

In [42]:
y_test_onehot = OneHotEncoder(sparse=False).fit_transform(y_test.to_numpy().reshape(-1, 1))
y_test_onehot.shape

(2947, 6)

In [58]:
predict_proba_diff = np.sum(np.absolute(y_test_onehot - y_pred_prob), axis=1)
predict_proba_diff.shape

(2947,)

In [59]:
predict_proba_diff

array([0., 0., 0., ..., 2., 2., 2.])

In [60]:
# Just sanity check if predict_proba_diff!=0 only contains items which y_test!=y_pred
((y_test!=y_pred) == (predict_proba_diff!=0)).all()

True

In [61]:
X_test[:, 0, :].shape

(2947, 561)

In [62]:
output_df = pd.DataFrame(X_test[:, 0, :], columns=feature_cols)
output_df['error'] = predict_proba_diff

In [63]:
output_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",error
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,0.0
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,0.0
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,0.0
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,0.0
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,0.0


In [64]:
output_df.to_csv(f'error_df.csv', index=False)